In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor


In [32]:
df = pd.read_csv('Sunspots.csv')


In [33]:
df.drop(columns='Unnamed: 0', inplace=True)
df.drop(index=df[df["Monthly Mean Total Sunspot Number"] == 0].index,inplace=True)

In [34]:
df['Date'] = pd.to_datetime(df['Date'])


In [35]:
df['Sunspot_lag1'] = df['Monthly Mean Total Sunspot Number'].shift(1)
df['Diff_11Year'] = df['Monthly Mean Total Sunspot Number'].diff(periods=11)
df['Diff_1Year']=df['Monthly Mean Total Sunspot Number'].diff(periods=1)
df['Month'] = df['Date'].dt.month
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
df['Year'] = df['Date'].dt.year


df.dropna(inplace=True)

In [ ]:
features = [
    'Sunspot_lag1',
    'Month',
    'Diff_11Year',
    'Diff_1Year',
    'Month_sin',
    'Month_cos',
    'Year',
    
]
X = df[features]
y = df['Monthly Mean Total Sunspot Number']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:


scaler = MinMaxScaler()
X_train_knn = scaler.fit_transform(X_train)
X_test_knn = scaler.transform(X_test)

model_knn = KNeighborsRegressor(n_neighbors=5)
model_knn.fit(X_train_knn, y_train)
y_pred_knn = model_knn.predict(X_test_knn)

In [39]:
y_pred= model_knn.predict(X_test_knn)

In [40]:
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [41]:
print("🔹 KNN")
print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")


🔹 KNN
R² Score: 0.8978
MSE: 473.36
MAE: 14.90


In [44]:
import matplotlib.pyplot as plt

# 🔹 Tarih bilgilerini çek
date_test = df.iloc[y_test.index]['Date'].values

# 🔹 Tahmin edilen son %20'lik kısmı al
last_20_index = int(len(y_test) * 0.2)
y_test_last = y_test[-last_20_index:]
y_pred_last = y_pred[-last_20_index:]
date_test_last = date_test[-last_20_index:]

# 🔹 Grafik çizimi
plt.figure(figsize=(14, 6))
plt.plot(date_test, y_test, label='Actual Sunspots', color='black', alpha=0.5)
plt.plot(date_test_last, y_pred_last, label='KNN Prediction (last 20%)', color='blue')
plt.xlabel('Date')
plt.ylabel('Monthly Mean Sunspot Number')
plt.title('KNN Prediction vs Actual Sunspots')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


IndexError: positional indexers are out-of-bounds